In [1]:
#@title Установка библиотек
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 faiss-cpu==1.7.4 gspread oauth2client nltk pydantic==1.10.8
#!pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


In [2]:
#@title Импорт библиотек
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken
import urllib.request
from textwrap import fill
#from transliterate import translit, get_available_language_codes

In [3]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [4]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
#    ("####", "Header 4"),
#    ("#####", "Header 5"),
#    ("######", "Header 6"),
                          ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)


    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:

        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment.page_content)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment.page_content}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks  и передача в чанк метада из маркдауновскго сплиттера
        source_chunks.extend(Document(page_content=chunk, metadata=fragment.metadata) for chunk in splitter.split_text(fragment.page_content))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    source_chunks = []
    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)


In [11]:
verbose = 0
max_count = 1800     #600
#count_type = "words"
count_type=''
top_similar_documents = 5

In [12]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 1, verbose = verbose, top_similar_documents = 5):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    # транслитерируем message_content
#    message_content_trl = translit(message_content, 'ru', reversed=True)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
#      {"role": "system", "content": system + f"{message_content_trl}"},
      {"role": "user", "content": topic}


      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', insert_newlines(completion.choices[0].message.content))

    # return completion

In [13]:
# Документ "База знаний в формате МаркДаун"

#url = 'https://docs.google.com/document/d/13f7_ZhX8-j2CZPhb1wG74byJxBkxe30p44GZW5xzge0'  # БЗ Ивана
#url = https://docs.google.com/document/d/1lwqhem98gfz1Jcm-xaGzLFbvqSQpuz7Kz1mU6oTxQRk'.  # БЗ Ивана в моей редакции (без дубля и первой статьи)
url = 'https://docs.google.com/document/d/1FX10CeV3HW71U0f4PnptqZOeU5KX3n2gmrpXgQBg7dY/edit?usp=sharing'  # united EN
#url = 'https://docs.google.com/document/d/1F1Fzuo_tAJFl8oHuyB-JIZry1ZUdvH7YxZRQeibmwYU/edit?usp=sharing'  # БЗ по видео
manual_index = load_search_indexes(url, max_count=max_count, count_type=count_type)



 ===========================================: 
Количество токенов в документе : 157947
ЦЕНА запроса: 0.06317880000000001  $


In [14]:
# Документ "Промт"
promt_url = 'https://docs.google.com/document/d/1lDIv_kklKb5yJMf9bOi2wupw4Mx4nbx3sbp2ymKGH9E'  # en
#promt_url = 'https://docs.google.com/document/d/1uUursMUL2EL4UflpK4TtvyD7SX1LseKrf07RVNmtovc'   # Ru

# Extract the document ID from the URL
match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', promt_url)
if match_ is None:
    raise ValueError('Invalid Google Docs URL')
doc_id = match_.group(1)

# Download the document as plain text
response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
response.raise_for_status()
manual_chat_promt = response.text

In [15]:
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"


def create_completion(model, system, content, temperature):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": content}
    ]

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )

    return completion.choices[0].message.content

def answer_index1(system, topic, search_index, temperature=0, verbose=verbose):
    docs = search_index.similarity_search(topic, k=4)
    message_content = ' '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)])
    question_content = f"Документ с информацией для ответа клиенту: {message_content}\n\nВопрос клиента: \n{topic}"
    return fill(create_completion(MODEL_TURBO_0613, system, question_content, temperature))

def summarize_questions(dialog):
    content = "Суммаризируй следующий диалог ассистента отдела обслуживания клиентов и клиента: " + " ".join(dialog)
    return create_completion(MODEL_TURBO_0613, "Ты - ассистент, который умеет профессионально суммаризировать присланные тебе диалоги. Твоя задача - суммаризировать диалог, который тебе пришел. Отражай имя клиента в саммаризации", content, 0)

def answer_user_question_dialog(system: str, db: str, user_question: str, question_history: list) -> str:
    summarized_history = ""
    if question_history:
        summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([f'{q} {a or ""}' for q, a in question_history])
    input_text = f"{summarized_history}\n\nТекущий вопрос: {user_question}"
    answer_text = answer_index(system, input_text, db)
    question_history.append((user_question, answer_text or ''))
    return fill(answer_text)


In [16]:
def run_dialog(system, db):
    question_history = []
    dialog = ""
    while True:
        user_question_q = input('Пользователь: ')
        if user_question_q.lower() == 'stop':
            break

        user_question = f'[Запрос пользователя]\n[{user_question_q}]'
        answer = answer_index(system, user_question, db)

        dialog += f'\nПользователь: {user_question_q} \nКонсультант: {answer}'

    #return dialog

In [ ]:
temperature=0.1
verbose = 1
run_dialog(manual_chat_promt, manual_index)

ОТВЕТ : 
  Чтобы передать метрики в системе Platrum, вам понадобится выполнить несколько шагов: 1. На вашей рабочей области у вас должны быть видны все доступные метрики, а также
 метрики для ваших подчиненных. Если у вас нет возможности видеть метрики подчиненных, пожалуйста, свяжитесь с администратором системы. 2. Для того чтобы заполнить свои
 метрики, перейдите на вкладку "Метрики" на вашей рабочей области и заполните соответствующие поля для каждой метрики. 3. Если у вас есть подчиненные, вы можете
 установить квоты для их метрик. Чтобы сделать это, перейдите на вкладку "Метрики", выберите нужного подчиненного и укажите квоты для его метрик. 4. Вы также можете
 управлять своими метриками, включая передачу их при уходе. Чтобы передать метрики при уходе из компании или изменении должности, обратитесь к администратору системы. Вот
 дополнительные ресурсы, которые могут быть полезны при работе с метриками: - [Создание метрик](https://www.youtube.com/watch?v=uxvrr7N-oSs&t=138.0s) - вид